In [2]:
from geopy.geocoders import Nominatim
from pyproj import Proj, Transformer
import geopandas as gpd
import numpy as np
import os

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi_exercise")

# List of attractions
attractions = [
    "Buckingham Palace, London",
    "Tower of London, London",
    "British Museum",
    "Big Ben, London",
    "London Eye, London"
]

# Getting the latitude and longitude using Geopy from OSM
lat_lon = []  # Initialize list to store lat/lon pairs
for attraction in attractions:
    location = geolocator.geocode(attraction)
    if location:
        lat_lon.append((location.latitude, location.longitude))
    else:
        print(f"Geocoding failed for: {attraction}")

# Reprojecting coordinates to BNG (EPSG:27700)
transformer = Transformer.from_crs("epsg:4326", "epsg:27700", always_xy=True)
eastings_northings = [transformer.transform(lon, lat) for lat, lon in lat_lon]

# Calculate midpoint
eastings, northings = zip(*eastings_northings)
midpoint = (np.mean(eastings), np.mean(northings))

# Load borough shapefile
cwd = os.getcwd()
boro_path = os.path.join(cwd, "data", "london-boundaries", "statistical-gis-boundaries-london", "ESRI", "London_Borough_Excluding_MHW.shp")  
boro = gpd.read_file(boro_path).to_crs('EPSG:27700')[['NAME', 'GSS_CODE', 'ONS_INNER', 'geometry']]

# Calculate centroids of boroughs and distance to midpoint
boro["centroid"] = boro.geometry.centroid
boro["distance_to_midpoint"] = boro["centroid"].apply(
    lambda point: np.sqrt((point.x - midpoint[0])**2 + (point.y - midpoint[1])**2)
)

# Display results
print(boro[["NAME", "distance_to_midpoint"]])


                      NAME  distance_to_midpoint
0     Kingston upon Thames          17693.906452
1                  Croydon          16977.295606
2                  Bromley          19057.030248
3                 Hounslow          17839.883561
4                   Ealing          14907.365028
5                 Havering          24366.287352
6               Hillingdon          23081.757006
7                   Harrow          18514.158551
8                    Brent          11902.052608
9                   Barnet          13737.957891
10                 Lambeth           5969.107545
11               Southwark           4789.131950
12                Lewisham           9402.585263
13               Greenwich          12656.745238
14                  Bexley          18700.617415
15                 Enfield          16185.268112
16          Waltham Forest          12140.031486
17               Redbridge          16043.266711
18                  Sutton          16619.903898
19    Richmond upon 